# OGG -> WAV

In [5]:
import numpy as np
import soundfile as sf
import typing

# import Vokaturi
import sys
sys.path.append("open_vok/api")
import Vokaturi
Vokaturi.load("open_vok/lib/Vokaturi_mac.so")

import audiotranscode
import emoji

In [6]:
def get_sample(path_ogg: str) -> tuple([np.array, int]):
    """
    :param path_ogg: path to .ogg file
    :return: samples of audio file and its sample rate
    """
    path_wav = path_ogg[:-3] + 'wav'
    
    at = audiotranscode.AudioTranscode()    
    at.transcode(path_ogg, path_wav)
    
    (samples, sample_rate) = sf.read(path_wav)

    return samples, sample_rate

def emotion_wrapper(path_ogg: str) -> tuple([bool, dict]):
    """
    :param path_ogg: path to .ogg file
    :return: indicator of validity and dictionary with detected
             emotions and its probabilities
    """
    (samples, sample_rate) = get_sample(path_ogg)
    buffer_length = len(samples)
    
    c_buffer = Vokaturi.SampleArrayC(buffer_length)
    c_buffer[:] = samples[:]

    voice = Vokaturi.Voice(sample_rate, buffer_length)

    # filling Voice with `samples`
    voice.fill(buffer_length, c_buffer)
    quality = Vokaturi.Quality()
    emotionProbabilities = Vokaturi.EmotionProbabilities()
    voice.extract(quality, emotionProbabilities)
    voice.destroy()

    quality.num_frames_analyzed
    
    emo_dict = {"neutrality": emotionProbabilities.neutrality,
                "happiness": emotionProbabilities.happiness,
                "sadness": emotionProbabilities.sadness,
                "anger": emotionProbabilities.anger,
                "fear": emotionProbabilities.fear}

    return quality.valid, emo_dict

In [7]:
def with_emoji(emo_dict: dict, mapping: dict) -> list:
    """
    :param emo_dict:
    :param mapping:
    :return:
    """
    text = []

    for emo, prob in emo_dict.items():
        text.append(emoji.emojize('{emo_str}: {prob_str:.3f}'.format(emo_str=mapping[emo],
                                                                 prob_str=prob), use_aliases=True))
        
    return text

# Work with test.wav

In [8]:
if quality.valid:
    print(emoji.emojize(':neutral_face: %.3f', use_aliases=True) % emotionProbabilities.neutrality)
    print(emoji.emojize(":smile: %.3f", use_aliases=True) % emotionProbabilities.happiness)
    print(emoji.emojize(":pensive: %.3f", use_aliases=True) % emotionProbabilities.sadness)
    print(emoji.emojize(":astonished: %.3f", use_aliases=True) % emotionProbabilities.anger)
    print(emoji.emojize(":scream: %.3f", use_aliases=True) % emotionProbabilities.fear)
else:
    print ("Not enough sonorancy to determine emotions")

NameError: name 'quality' is not defined

In [9]:
emotion_wrapper('wow_sure.ogg')

OSError: File not Found! Cannot decode "file" /Users/AntonKarazeev/WD/neurohack/SenseOfSpeech/wow_sure.ogg

In [10]:
emo_mapping = {
    'anger': ':astonished:',
    'fear': ':scream:',
    'happiness': ':smile:',
    'neutrality': ':neutral_face:',
    'sadness': ':pensive:'
}

In [11]:
valid, emo_dict = emotion_wrapper('wow_sure.ogg')

OSError: File not Found! Cannot decode "file" /Users/AntonKarazeev/WD/neurohack/SenseOfSpeech/wow_sure.ogg

In [12]:
emo_dict

NameError: name 'emo_dict' is not defined

In [13]:
if valid:
    print('\n'.join(with_emoji(emo_dict, emo_mapping)))
else:
    err_message = emoji.emojize("¯\_(ツ)_/¯", use_aliases=True)
    print(err_message)

NameError: name 'valid' is not defined

In [14]:
emo_dict

NameError: name 'emo_dict' is not defined

In [15]:
testdict = {'a': 1, 'b': 2}

In [16]:
dict(zip(testdict.values(), testdict.keys()))

{1: 'a', 2: 'b'}

# Let's GO

In [17]:
from util import logger, emotion_file_path, emo_distribution, to_text
from accessories import actions, rev_actions, rev_mapping

In [25]:
emotion = 'anger'
valid, emo_dict1 = emo_distribution(emotion_file_path(emotion))
emo_dict1

{'anger': 0.9995287435466227,
 'fear': 6.572438958602349e-10,
 'happiness': 2.0288595201133144e-05,
 'neutrality': 0.0004195648455337856,
 'sadness': 3.140235539847084e-05}

In [27]:
emotion = 'fear'
valid, emo_dict2 = emo_distribution(emotion_file_path(emotion))
emo_dict2

{'anger': 0.0015831422289514314,
 'fear': 0.9755983599427229,
 'happiness': 0.0069784240971001825,
 'neutrality': 0.01524911470212832,
 'sadness': 0.0005909590290971136}

In [36]:
sorted(emo_dict1.items(), key=lambda x: x[0])

[('anger', 0.9995287435466227),
 ('fear', 6.572438958602349e-10),
 ('happiness', 2.0288595201133144e-05),
 ('neutrality', 0.0004195648455337856),
 ('sadness', 3.140235539847084e-05)]

In [46]:
emotions1 = np.array(list(map(lambda x: x[1], sorted(emo_dict1.items(), key=lambda x: x[0]))))
emotions1

array([  9.99528744e-01,   6.57243896e-10,   2.02885952e-05,
         4.19564846e-04,   3.14023554e-05])

In [47]:
emotions2 = np.array(list(map(lambda x: x[1], sorted(emo_dict2.items(), key=lambda x: x[0]))))
emotions2

array([  1.58314223e-03,   9.75598360e-01,   6.97842410e-03,
         1.52491147e-02,   5.90959029e-04])

In [53]:
sum((emotions1 - emotions2) ** 2)

1.9479562261320902

In [63]:
def emo_distance(emo_dict1, emo_dict2):
    def get_array(emo_dict):
        return np.array(list(map(lambda x: x[1], sorted(emo_dict.items(), key=lambda x: x[0]))))

    emotions1 = get_array(emo_dict1)
    emotions2 = get_array(emo_dict2)

    tmp_sum = sum((emotions1 - emotions2) ** 2)

    return np.sqrt(tmp_sum)

In [64]:
emo_distance(emo_dict1, emo_dict2)

1.3956920240984723

In [65]:
emoji.emojize("kek :thumbsup: lol", use_aliases=True)

'kek 👍 lol'

In [71]:
emoji.emojize("Well done :thumbsup:", use_aliases=True)

'Well done 👍'

In [70]:
emoji.emojize("It's ok :ok_hand:", use_aliases=True)

"It's ok 👌"

In [69]:
emoji.emojize("Keep trying :sweat_smile:", use_aliases=True)

'Keep trying 😅'